# ============================================
# TELCO CHURN BUSINESS INSIGHTS DASHBOARD
# ============================================


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ============================================
# 1. LOAD DATA
# ============================================

In [2]:
df = pd.read_csv(r'C:\Users\ADMIN\Desktop\DataAnalytics\customer-churn-prediction\data\processed\cleaned_customer_churn.csv')

# Data cleaning
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges'])
df['tenure'] = pd.to_numeric(df['tenure'])
df['MonthlyCharges'] = pd.to_numeric(df['MonthlyCharges'])
print(df.head(3))

   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   

      PaymentMethod MonthlyCharges  TotalCharges  Churn  
0  

# ============================================
# 2. TOP KPIs CALCULATION
# ============================================

In [3]:
total_customers = len(df)
churned_customers = len(df[df['Churn'] == 'Yes'])
churn_rate = (churned_customers / total_customers) * 100

# Annual Revenue at Risk
annual_revenue_at_risk = df[df['Churn'] == 'Yes']['MonthlyCharges'].sum() * 12

# Average Revenue Per Churned Customer
avg_revenue_churned = df[df['Churn'] == 'Yes']['MonthlyCharges'].mean()

# Preventable Churn (customers in month-to-month contracts)
preventable_churn = len(df[(df['Churn'] == 'Yes') & (df['Contract'] == 'Month-to-month')])
preventable_churn_pct = (preventable_churn / churned_customers) * 100

print("=" * 60)
print("KEY PERFORMANCE INDICATORS")
print("=" * 60)
print(f"Total Customers: {total_customers:,}")
print(f"Churn Rate: {churn_rate:.2f}%")
print(f"Annual Revenue at Risk: ${annual_revenue_at_risk:,.2f}")
print(f"Avg Monthly Revenue per Churned Customer: ${avg_revenue_churned:.2f}")
print(f"Preventable Churn (Month-to-month): {preventable_churn_pct:.2f}%")
print("=" * 60)

KEY PERFORMANCE INDICATORS
Total Customers: 7,043
Churn Rate: 26.54%
Annual Revenue at Risk: $1,669,570.20
Avg Monthly Revenue per Churned Customer: $74.44
Preventable Churn (Month-to-month): 88.55%


# ============================================
# 3. CHART 1: REVENUE LOSS BY CONTRACT TYPE
# ============================================

In [4]:
contract_analysis = df.groupby(['Contract', 'Churn']).agg({
    'MonthlyCharges': 'sum',
    'customerID': 'count'
}).reset_index()
contract_analysis.columns = ['Contract', 'Churn', 'Monthly_Revenue', 'Customer_Count']
contract_analysis['Annual_Revenue'] = contract_analysis['Monthly_Revenue'] * 12

fig1 = px.bar(contract_analysis, 
              x='Contract', 
              y='Annual_Revenue',
              color='Churn',
              title='Revenue Loss by Contract Type (Annual)',
              labels={'Annual_Revenue': 'Annual Revenue ($)', 'Contract': 'Contract Type'},
              color_discrete_map={'Yes': '#EF553B', 'No': '#00CC96'},
              barmode='stack',
              text_auto='.2s')

fig1.update_layout(
    plot_bgcolor='white',
    height=500,
    showlegend=True,
    legend_title='Churned'
)
fig1.show()

# Calculate churn rate by contract type
churn_by_contract = df.groupby('Contract')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100).reset_index()
churn_by_contract.columns = ['Contract', 'Churn_Rate']
print("\nChurn Rate by Contract Type:")
print(churn_by_contract.to_string(index=False))


Churn Rate by Contract Type:
      Contract  Churn_Rate
Month-to-month   42.709677
      One year   11.269518
      Two year    2.831858


# ============================================
# 4. CHART 2: CHURN RATE BY TENURE (0-24 MONTHS)
# ============================================

In [5]:
tenure_df = df[df['tenure'] <= 24].copy()
tenure_churn = tenure_df.groupby('tenure')['Churn'].apply(lambda x: (x == 'Yes').mean() * 100).reset_index()
tenure_churn.columns = ['Tenure_Months', 'Churn_Rate']

fig2 = px.line(tenure_churn, 
               x='Tenure_Months', 
               y='Churn_Rate',
               title='Churn Rate by Tenure (First 24 Months)',
               labels={'Tenure_Months': 'Months as Customer', 'Churn_Rate': 'Churn Rate (%)'},
               markers=True)

fig2.add_hline(y=churn_rate, line_dash="dash", line_color="red", 
               annotation_text=f"Overall Churn Rate: {churn_rate:.1f}%")

fig2.update_traces(line_color='#636EFA', line_width=3)
fig2.update_layout(
    plot_bgcolor='white',
    height=500,
    showlegend=False
)
fig2.show()

print("\nHighest Risk Tenure Periods:")
print(tenure_churn.nlargest(5, 'Churn_Rate').to_string(index=False))


Highest Risk Tenure Periods:
 Tenure_Months  Churn_Rate
             1   61.990212
             2   51.680672
             5   48.120301
             4   47.159091
             3   47.000000


# ============================================
# 5. CHART 3: CHURN BY PAYMENT METHOD
# ============================================

In [6]:
payment_churn = df.groupby('PaymentMethod').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'customerID': 'count'
}).reset_index()
payment_churn.columns = ['PaymentMethod', 'Churn_Rate', 'Customer_Count']
payment_churn = payment_churn.sort_values('Churn_Rate', ascending=False)

fig3 = px.bar(payment_churn,
              x='Churn_Rate',
              y='PaymentMethod',
              orientation='h',
              title='Churn Rate by Payment Method',
              labels={'Churn_Rate': 'Churn Rate (%)', 'PaymentMethod': 'Payment Method'},
              color='Churn_Rate',
              color_continuous_scale='Reds',
              text_auto='.1f')

fig3.update_layout(
    plot_bgcolor='white',
    height=400,
    showlegend=False
)
fig3.show()

print("\nChurn Rate by Payment Method:")
print(payment_churn.to_string(index=False))


Churn Rate by Payment Method:
            PaymentMethod  Churn_Rate  Customer_Count
         Electronic check   45.285412            2365
             Mailed check   19.106700            1612
Bank transfer (automatic)   16.709845            1544
  Credit card (automatic)   15.243101            1522


# ============================================
# 6. CHART 4: SERVICE BUNDLE IMPACT
# ============================================

In [9]:
# Count number of add-on services
service_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'StreamingTV', 'StreamingMovies']

df['Service_Count'] = 0
for col in service_columns:
    df['Service_Count'] += (df[col] == 'Yes').astype(int)

# Create service segments
df['Service_Segment'] = pd.cut(df['Service_Count'], 
                                bins=[-1, 0, 2, 6], 
                                labels=['No Services', '1-2 Services', '3+ Services'])

bundle_analysis = df.groupby('Service_Segment').agg({
    'Churn': lambda x: (x == 'Yes').mean() * 100,
    'customerID': 'count',
    'MonthlyCharges': 'mean'
}).reset_index()
bundle_analysis.columns = ['Service_Segment', 'Churn_Rate', 'Customer_Count', 'Avg_Monthly_Charge']

fig4 = px.bar(bundle_analysis,
              x='Service_Segment',
              y='Churn_Rate',
              title='Churn Rate by Service Bundle Size',
              labels={'Service_Segment': 'Service Bundle', 'Churn_Rate': 'Churn Rate (%)'},
              color='Churn_Rate',
              color_continuous_scale='RdYlGn_r',
              text_auto='.1f')

fig4.update_layout(
    plot_bgcolor='white',
    height=500,
    showlegend=False
)
fig4.show()

print("\nService Bundle Analysis:")
print(bundle_analysis.to_string(index=False))


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_688\3162876378.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




Service Bundle Analysis:
Service_Segment  Churn_Rate  Customer_Count  Avg_Monthly_Charge
    No Services   21.406039            2219           32.792902
   1-2 Services   40.620310            1999           69.110555
    3+ Services   20.601770            2825           86.795451


# ============================================
# 7. CHART 5: HIGH-VALUE CUSTOMER RISK SEGMENTS
# ============================================

In [10]:
# Focus on customers with less than 24 months tenure
risk_df = df[df['tenure'] <= 24].copy()

fig5 = px.scatter(risk_df,
                  x='tenure',
                  y='MonthlyCharges',
                  color='Churn',
                  title='High-Value Customer Risk Analysis (First 24 Months)',
                  labels={'tenure': 'Months as Customer', 
                         'MonthlyCharges': 'Monthly Charges ($)',
                         'Churn': 'Churned'},
                  color_discrete_map={'Yes': '#EF553B', 'No': '#00CC96'},
                  opacity=0.6,
                  size_max=10)

fig5.update_layout(
    plot_bgcolor='white',
    height=600,
    showlegend=True
)
fig5.show()

# High-value at-risk customers (>$80/month, <12 months tenure, churned)
high_value_risk = df[(df['MonthlyCharges'] > 80) & 
                     (df['tenure'] < 12) & 
                     (df['Churn'] == 'Yes')]

print(f"\nHigh-Value At-Risk Customers (>$80/month, <12 months):")
print(f"Count: {len(high_value_risk)}")
print(f"Annual Revenue Lost: ${(high_value_risk['MonthlyCharges'].sum() * 12):,.2f}")


High-Value At-Risk Customers (>$80/month, <12 months):
Count: 321
Annual Revenue Lost: $347,775.60


# ============================================
# 8. ACTIONABLE INSIGHTS SUMMARY
# ============================================

In [11]:
print("\n" + "=" * 60)
print("ACTIONABLE BUSINESS INSIGHTS")
print("=" * 60)

# Insight 1: Contract type impact
mtm_churn_rate = df[df['Contract'] == 'Month-to-month']['Churn'].apply(lambda x: x == 'Yes').mean() * 100
ltc_churn_rate = df[df['Contract'] != 'Month-to-month']['Churn'].apply(lambda x: x == 'Yes').mean() * 100
print(f"\n1. CONTRACT TYPE:")
print(f"   - Month-to-month churn: {mtm_churn_rate:.1f}%")
print(f"   - Long-term contract churn: {ltc_churn_rate:.1f}%")
print(f"   - ACTION: Offer 15% discount for 1-year commitments")
print(f"   - POTENTIAL IMPACT: Reduce {preventable_churn:,} churns")

# Insight 2: Payment method
echeck_churn = df[df['PaymentMethod'] == 'Electronic check']['Churn'].apply(lambda x: x == 'Yes').mean() * 100
other_churn = df[df['PaymentMethod'] != 'Electronic check']['Churn'].apply(lambda x: x == 'Yes').mean() * 100
print(f"\n2. PAYMENT METHOD:")
print(f"   - Electronic check churn: {echeck_churn:.1f}%")
print(f"   - Auto-pay methods churn: {other_churn:.1f}%")
print(f"   - ACTION: Incentivize auto-pay adoption ($5/month discount)")

# Insight 3: Service bundles
no_service_churn = df[df['Service_Count'] == 0]['Churn'].apply(lambda x: x == 'Yes').mean() * 100
multi_service_churn = df[df['Service_Count'] >= 3]['Churn'].apply(lambda x: x == 'Yes').mean() * 100
print(f"\n3. SERVICE ENGAGEMENT:")
print(f"   - No add-on services churn: {no_service_churn:.1f}%")
print(f"   - 3+ services churn: {multi_service_churn:.1f}%")
print(f"   - ACTION: Bundle promotions in first 30 days")

# Insight 4: Early tenure risk
early_churn = df[df['tenure'] <= 6]['Churn'].apply(lambda x: x == 'Yes').mean() * 100
print(f"\n4. ONBOARDING:")
print(f"   - First 6 months churn rate: {early_churn:.1f}%")
print(f"   - ACTION: Implement 90-day onboarding program")

print("\n" + "=" * 60)
print("ESTIMATED ANNUAL REVENUE PROTECTION: $" + f"{annual_revenue_at_risk * 0.3:,.2f}")
print("(Assuming 30% reduction in churn from these initiatives)")
print("=" * 60)


ACTIONABLE BUSINESS INSIGHTS

1. CONTRACT TYPE:
   - Month-to-month churn: 42.7%
   - Long-term contract churn: 6.8%
   - ACTION: Offer 15% discount for 1-year commitments
   - POTENTIAL IMPACT: Reduce 1,655 churns

2. PAYMENT METHOD:
   - Electronic check churn: 45.3%
   - Auto-pay methods churn: 17.1%
   - ACTION: Incentivize auto-pay adoption ($5/month discount)

3. SERVICE ENGAGEMENT:
   - No add-on services churn: 21.4%
   - 3+ services churn: 20.6%
   - ACTION: Bundle promotions in first 30 days

4. ONBOARDING:
   - First 6 months churn rate: 52.9%
   - ACTION: Implement 90-day onboarding program

ESTIMATED ANNUAL REVENUE PROTECTION: $500,871.06
(Assuming 30% reduction in churn from these initiatives)
